In [1]:
%load_ext autoreload
%autoreload 2

import os
import oddsapi
import requests
import joblib
import pandas as pd
import numpy as np
from common.utils.odds_utils import get_tournament_winner_odds, get_group_stage_odds, get_group_stage_OU
from common.utils.holdet_utils import get_holdet_data, get_play_off_opps, get_rw
from common.utils.simulation_utils import WorldCupTeam, WorldCupSim
from common.utils.group_stage import get_group_stage, playoffs
from common.utils.dict_utils import eng2dan
from scipy.optimize import fsolve
import warnings
warnings.filterwarnings('ignore')
ROUND = 2
API_KEY = os.getenv("API_KEY")

In [2]:
# load data
df_holdet = get_holdet_data()
df_winner = get_tournament_winner_odds(API_KEY)
df_group_stage = get_group_stage_odds(API_KEY)
df_get_group_stage_OU = get_group_stage_OU(API_KEY)
df_groups = pd.read_csv("common/ressources/df_groups.csv")[["Team","group"]]
df_group_stage = df_group_stage[['home_team','away_team','prob1','probx', 'prob2']].merge(df_groups.rename(columns =  {"Team":"home_team"}), on = "home_team")

In [3]:
_df_group_stage = get_group_stage(df_get_group_stage_OU.copy(), df_group_stage.copy(), ROUND, eng2dan)

In [17]:
_df_group_stage

,home_team,away_team,prob1,probx,prob2,group,xG,xGH,xGA,round
0,Wales,Iran,0.440286,0.310952,0.248762,B,3.117121,1.991771,1.125350,2
1,Portugal,Uruguay,0.498575,0.282051,0.219373,H,2.948197,2.047359,0.900838,2
2,Brasilien,Schweiz,0.682816,0.202058,0.115126,G,2.456675,2.102229,0.354446,2
3,Sydkorea,Ghana,0.371983,0.311536,0.316481,H,2.571368,1.389333,1.182035,2
4,Cameroun,Serbien,0.176849,0.257235,0.565916,G,2.837655,0.675632,2.162023,2
5,Spanien,Tyskland,0.404401,0.266686,0.328913,E,2.471358,1.362881,1.108477,2
6,Belgien,Marokko,0.487239,0.276102,0.236659,F,2.826512,1.902460,0.924052,2
7,Japan,Costa Rica,0.672671,0.214962,0.112367,E,2.777084,2.379585,0.397499,2
8,Argentina,Mexico,0.617736,0.241068,0.141197,C,2.793940,2.274137,0.519803,2
9,Frankrig,Danmark,0.550100,0.265515,0.184385,D,2.904052,2.175018,0.729034,2


In [4]:
df_rw = get_rw(df_holdet)
df_rw.rename(columns = {"Player_match": "name1"}, inplace = True)
print(df_holdet.shape)
df_holdet = df_holdet.merge(df_rw, on = "name1", how ="left")
print(df_holdet.shape)
df_holdet = df_holdet[['name1', 'name2', 'country', 'price', 'position', 'Player', 'Team', 'Opp', 'Pos', 'MIN','G', 'A', 'SOG', 'Y','R']]
print(df_holdet.shape)

(831, 10)
(831, 39)
(831, 15)


In [75]:
# df_holdet.loc[df_holdet["name1"]=="Martin Braithwaite","G"]=0.3
# df_holdet.loc[df_holdet["country"]=="Danmark",:]

In [5]:
for round in [str(_) for _ in range(ROUND,4)]:
    df_temp1 = df_holdet[["name1","name2","country"]].merge(_df_group_stage.loc[_df_group_stage["round"]==round,["home_team","prob1","probx","prob2","xGH","xGA"]].rename(columns = {"home_team":"country"}), on = "country")
    df_temp1.rename(columns = {"prob1":f"prob_winning_{round}","probx":f"prob_draw_{round}","prob2":f"prob_lose_{round}","xGH":f"E_goals_{round}","xGA":f"E_goals_against_{round}"}, inplace = True)

    df_temp2 = df_holdet[["name1","name2","country"]].merge(_df_group_stage.loc[_df_group_stage["round"]==round,["away_team","prob1","probx","prob2","xGH","xGA"]].rename(columns = {"away_team":"country"}), on = "country")
    df_temp2.rename(columns = {"prob1":f"prob_lose_{round}","probx":f"prob_draw_{round}","prob2":f"prob_winning_{round}","xGA":f"E_goals_{round}","xGH":f"E_goals_against_{round}"}, inplace = True)

    df_temp_temp = pd.concat([df_temp1,df_temp2],axis=0)
    df_holdet = df_holdet.merge(df_temp_temp, on = ["name1","name2","country"])

In [10]:
df_winner  = playoffs(df_winner.copy(),_df_group_stage.copy(), eng2dan)
df_holdet = df_holdet.merge(df_winner[['country','R16_prob', 'QF_prob', 'SF_prob', 'Final_prob', 'W_prob', \
                                       'prob_to_win_R16', 'prob_to_win_QF', 'prob_to_win_SF','prob_to_win_Final', \
                                       'xG_R16', 'xG_QF', 'xG_SF', 'xG_Final','xGa_R16', 'xGa_QF', 'xGa_SF', 'xGa_Final']], on = "country")

df_holdet["selected"] = ~df_holdet['G'].isna()

In [11]:
for country in df_holdet["country"].unique():
    xG_sum = df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'G'].astype(float).sum()
    xA_sum = df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'A'].astype(float).sum()
    df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'G_share'] = df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'G'].astype(float)/xG_sum
    df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'A_share'] = df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'A'].astype(float)/xA_sum

    df_holdet.loc[(df_holdet["country"]==country) & (~df_holdet["selected"]),"E_points_total"] = -5000
    df_holdet.loc[(df_holdet["country"]==country) & (~df_holdet["selected"]),"E_points_round1"] = -5000
    for player in df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),"name1"]:
        points = 0
        goal_share = df_holdet.loc[df_holdet["name1"]==player,"G_share"].values[0]
        assist_share = df_holdet.loc[df_holdet["name1"]==player,"A_share"].values[0]
        yellow_prob = df_holdet.loc[df_holdet["name1"]==player,'Y'].astype(float).values[0]
        red_prob = df_holdet.loc[df_holdet["name1"]==player,'R'].astype(float).values[0]
        # group games
        loop = 4
        for i in range(ROUND,loop):
            points += 7000*(1/(i-ROUND+1))
            points += df_holdet.loc[df_holdet["name1"]==player,f'prob_winning_{i}'].values[0]*25000*(1/(i-ROUND+1))
            points += df_holdet.loc[df_holdet["name1"]==player,f'prob_draw_{i}'].values[0]*5000*(1/(i-ROUND+1))
            points += df_holdet.loc[df_holdet["name1"]==player,f'prob_lose_{i}'].values[0]*-15000*(1/(i-ROUND+1))
            points += df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*10000*(1/(i-ROUND+1))
            points += df_holdet.loc[df_holdet["name1"]==player,f'E_goals_against_{i}'].values[0]*-10000*(1/(i-ROUND+1))
            points += yellow_prob * -20000*(1/(i-ROUND+1))
            points += yellow_prob*yellow_prob*-20000*(1/(i-ROUND+1))
            points += red_prob * -50000*(1/(i-ROUND+1))
            if i == loop:
                for idx, game in enumerate(["R16","QF","SF","Final"]):
                    prob = df_holdet.loc[df_holdet["name1"]==player,f'{game}_prob'].values[0]
                    factor = 1/(2*(idx))
                    points += 7000*factor
                    points += prob*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xGa_{game}'].values[0])*-10000*factor
                    points += prob*df_holdet.loc[df_holdet["name1"]==player,f'prob_to_win_{game}'].values[0]*25000*factor
                    points += prob*(1-df_holdet.loc[df_holdet["name1"]==player,f'prob_to_win_{game}'].values[0])*-15000*factor
                    points += prob*yellow_prob * -20000*factor
                    points += prob*yellow_prob*yellow_prob*-20000*factor
                    points += prob*df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0]*10000*factor
                    points += prob*red_prob * -50000*factor
                    
            if df_holdet.loc[df_holdet["name1"]==player,"position"].values[0]=="Mål":
                points += np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'E_goals_against_{i}'].values[0])*75000*(1/(i-ROUND+1))
                if i == loop:
                    for idx, game in enumerate(["R16","QF","SF","Final"]):
                        prob = df_holdet.loc[df_holdet["name1"]==player,f'{game}_prob'].values[0]
                        factor = 1/(2*(idx))
                        points += prob*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xGa_{game}'].values[0])*75000*factor
                        
            elif df_holdet.loc[df_holdet["name1"]==player,"position"].values[0]=="Forsvar":
                points += np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'E_goals_against_{i}'].values[0])*50000*(1/(i-ROUND+1))
                points += goal_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*175000*(1/(i-ROUND+1))
                points += assist_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*60000*(1/(i-ROUND+1))
                if i == loop:
                    for idx, game in enumerate(["R16","QF","SF","Final"]):
                        prob = df_holdet.loc[df_holdet["name1"]==player,f'{game}_prob'].values[0]
                        factor = 1/(2*(idx))
                        points += prob*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*50000*factor
                        points += prob*goal_share*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*175000*factor
                        points += prob*assist_share*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*60000*factor
                        
            elif df_holdet.loc[df_holdet["name1"]==player,"position"].values[0]=="Midtbane":
                points += goal_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*150000*(1/(i-ROUND+1))
                points += assist_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*60000*(1/(i-ROUND+1))
                if i == loop:
                    for idx, game in enumerate(["R16","QF","SF","Final"]):
                        prob = df_holdet.loc[df_holdet["name1"]==player,f'{game}_prob'].values[0]
                        factor = 1/(2*(idx))
                        points += prob*goal_share*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*150000*factor
                        points += prob*assist_share*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*60000*factor
                        
            elif df_holdet.loc[df_holdet["name1"]==player,"position"].values[0]=="Angreb":
                points += goal_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*125000*(1/(i-ROUND+1))
                points += assist_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*60000*(1/(i-ROUND+1))
                if i == loop:
                    for idx, game in enumerate(["R16","QF","SF","Final"]):
                        prob = df_holdet.loc[df_holdet["name1"]==player,f'{game}_prob'].values[0]
                        factor = 1/(2*(idx))
                        points += prob*goal_share*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*125000*factor
                        points += prob*assist_share*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*60000*factor
            if i == ROUND:
                df_holdet.loc[df_holdet["name1"]==player,"E_points_round1"] = points
        df_holdet.loc[df_holdet["name1"]==player,"E_points_total"] = points

In [18]:
df_holdet.to_csv("common/ressources/df_holdet", index = False)


In [16]:
df_holdet = pd.read_csv("common/ressources/df_holdet")

In [17]:
df_holdet.loc[df_holdet["country"]=="Senegal","E_points_total"] = 0
df_holdet.loc[df_holdet["country"]=="Ghana","E_points_total"] = 0
df_holdet.loc[df_holdet["country"]=="Japan","E_points_total"] = 0
df_holdet.loc[df_holdet["country"]=="Senegal","E_points_round1"] = 0
df_holdet.loc[df_holdet["country"]=="Ghana","E_points_round1"] = 0
df_holdet.loc[df_holdet["country"]=="Japan","E_points_round1"] = 0


In [15]:
df_holdet.sort_values("E_points_total", ascending = False).iloc[0:40]

,name1,name2,country,price,position,Player,Team,Opp,Pos,MIN,...,xG_Final,xGa_R16,xGa_QF,xGa_SF,xGa_Final,selected,G_share,A_share,E_points_total,E_points_round1
0,Kylian Mbappe,K. Mbappe,Frankrig,9282000.0,Angreb,Kylian Mbappe,FRA,AUS,F,90.0,...,1.593750,0.367617,0.594549,0.483353,0.367617,True,0.354167,0.229167,247019.777675,158008.516822
77,Lionel Messi,L. Messi,Argentina,8131000.0,Angreb,Lionel Messi,ARG,KSA,F/M,90.0,...,1.920209,0.575312,0.533931,0.408978,0.408978,True,0.318339,0.232804,242046.078613,160925.202129
207,Robert Lewandowski,R. Lewandowski,Polen,6482000.0,Angreb,Robert Lewandowski,POL,MEX,F,90.0,...,0.090019,0.518865,0.223467,0.065375,0.029840,True,0.625000,0.184466,234414.335680,223745.799464
108,Cody Gakpo,C. Gakpo,Holland,4730000.0,Angreb,Cody Gakpo,NED,SEN,F/M,90.0,...,0.500893,0.133422,0.160107,0.242267,0.160107,True,0.262443,0.286585,234277.793622,135679.584228
51,Neymar Jr.,N. Jr.,Brasilien,9062000.0,Angreb,Neymar,BRA,SRB,F/M,90.0,...,1.766713,0.288051,0.253385,0.217046,0.217046,True,0.282297,0.227586,207527.430854,136237.481856
25,Harry Kane,H. Kane,England,9194000.0,Angreb,Harry Kane,ENG,IRN,F,90.0,...,1.520202,0.519206,0.318721,0.420607,0.318721,True,0.290456,0.149425,204010.560354,132442.922911
104,Denzel Dumfries,D. Dumfries,Holland,5602000.0,Forsvar,Denzel Dumfries,NED,SEN,M/D,90.0,...,0.500893,0.133422,0.160107,0.242267,0.160107,True,0.099548,0.146341,197384.894336,111818.451542
155,Kevin De Bruyne,K. De Bruyne,Belgien,7585000.0,Midtbane,Kevin De Bruyne,BEL,CAN,F/M,90.0,...,0.848258,1.728358,0.226684,0.533374,0.533374,True,0.211618,0.471264,193261.571767,139706.675599
78,Lautaro Martinez,L. Martinez,Argentina,7486000.0,Angreb,Lautaro Martinez,ARG,KSA,F,90.0,...,1.920209,0.575312,0.533931,0.408978,0.408978,True,0.273356,0.084656,190247.555644,126369.544641
267,Marco Asensio,M. Asensio,Spanien,4722000.0,Midtbane,Marco Asensio,ESP,CRC,F/M,90.0,...,1.346297,0.714120,0.287064,0.534727,0.534727,True,0.301818,0.210227,176851.067935,94328.034892
